In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



C:\Users\y.bukhar\AppData\Local\Temp\ipykernel_6376\2658766297.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Библотеки и связ с БД

In [2]:
# from sqlalchemy import create_engine
import requests as req
import pandas as pd
import numpy as np
import pyodbc
import pypyodbc

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

conn = pypyodbc.connect('Driver=SQL Server;Server=192.168.1.209;Database=AA_DWH_X;Trusted_Connection=yes;')
cursor = conn.cursor()

#  Область като и кадастр

In [6]:
r = req.post('https://aisgzk.kz/aisgzk/Index/BuildRegionTree', json={"key": "value"})
r=r.json()
data=r['data']
DF_OBL=pd.DataFrame()
for i in data:
    OBL=pd.json_normalize(i)
    DF_OBL=pd.concat([DF_OBL,OBL])
DF_OBL=DF_OBL.drop(['naimpre','type','iconCls','expanded','children'],axis=1)
DF_OBL

,text,klkod,vnaim,kato
0,Акмолинская,01,область,110000000
0,Актюбинская,02,область,150000000
0,Алматинская,03,область,190000000
0,Алматы,20,город республиканского значения,750000000
0,Атырауская,04,область,230000000
0,Восточно-Казахстанская,05,область,630000000
0,Жамбылская,06,область,310000000
0,Западно-Казахстанская,08,область,270000000
0,Карагандинская,09,область,350000000
0,Костанайская,12,область,390000000


# Tолько области

In [12]:
# ### это нужен для ГЕОМЕТРИ область
oblast=DF_OBL['text']
oblast

0               Акмолинская
0               Актюбинская
0               Алматинская
0                    Алматы
0                Атырауская
0    Восточно-Казахстанская
0                Жамбылская
0     Западно-Казахстанская
0            Карагандинская
0              Костанайская
0            Кызылординская
0             Мангистауская
0                Нур-Султан
0             Павлодарская 
0      Северо-Казахстанская
0             Туркестанская
0                   Шымкент
Name: text, dtype: object

# районы с кадастрывыми кодами Областя

In [35]:
r = req.post('https://aisgzk.kz/aisgzk/Index/BuildRegionTree', json={"key": "value"})
r=r.json()
data=r['data']
DF_OBL_RAION=pd.DataFrame()
for i in data:
    OBL=pd.json_normalize(i['children'])
    OBL["OBL"]=i['klkod']
    DF_OBL_RAION=pd.concat([DF_OBL_RAION,OBL])
DF_OBL_RAION

,text,klkod,vnaim,naimpre,kato,type,iconCls,expanded,leaf,OBL
0,Акколь,01|014,город областного значения,1,113220000,1,tree-last,True,True,01
1,Аккольский,01|001,район области или район в городе республиканск...,0,113200000,1,tree-last,True,True,01
2,Аршалынский,01|005,район области или район в городе республиканск...,0,113400000,1,tree-last,True,True,01
3,Астраханский,01|002,район области или район в городе республиканск...,0,113600000,1,tree-last,True,True,01
4,Атбасар,01|015,город областного значения,1,113820100,1,tree-last,True,True,01
...,...,...,...,...,...,...,...,...,...,...
22,Шымкент,19|309,город областного значения,1,790000000,1,tree-last,True,True,19
0,Абайский,22|327,район области или район в городе республиканск...,0,791110000,1,tree-last,True,True,22
1,Аль-Фарабийский,22|328,район области или район в городе республиканск...,0,791310000,1,tree-last,True,True,22
2,Енбекшинский,22|329,район области или район в городе республиканск...,0,791510000,1,tree-last,True,True,22


# районы layers_ID 

In [ ]:
# ### это нужен для парсера площади ТОО,КХ,ФЗ
layers='https://aisgzk.kz/aisgzk/Proxy/aisgzkZem2/MapServer?f=json&dpi=96&transparent=true&format=png8'
layers = req.get(layers, json={"key": "value"}).json()
layers = pd.json_normalize(layers['layers'])
layers['name']=layers['name'].str.replace("U_","")
layers['name']=layers['name'].str.replace("_","|")
layers=layers.drop(['parentLayerId','defaultVisibility','subLayerIds','minScale','maxScale'],axis=1)

UPDATE='''UPDATE [L0_X].[dbo].[AISGZK_RAION] SET LAYERS_ID = ? WHERE [KLKOD] = ?'''
try:
    conn.autocommit = False
    for t in layers.values.tolist():
        cursor.execute(UPDATE, t[0],t[1])
except Exception as e:
    print(t)
    cursor.rollback()
    print(e)
    print('transaction rolled back')
else:
    print('inserted successfully')
    conn.commit()
finally:
    conn.autocommit = True

# geometry Области

In [6]:
# ### geometry OBL
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


conn = pyodbc.connect('Driver=SQL Server;Server=192.168.1.209;Database=L0_X;Trusted_Connection=yes;')
cursor = conn.cursor()

#----------------------------------------------Get json------------------------------------------------------------------------

url = "https://aisgzk.kz/aisgzk/Proxy/aisgzkTopo2/MapServer/38/query?f=json&where=NAME_ADM1 like '%{}%'&returnGeometry=true&spatialRel=esriSpatialRelIntersects&outFields=*&outSR=3857"

#-------------------------------------------Get list of obl names from AIZGSK--------------------------------------------------#

request = req.post('https://aisgzk.kz/aisgzk/Index/BuildRegionTree', json={"key": "value"})
request = request.json()
data=request['data']
obl = []
for k in data:
    obl.append(k['text'])
#----------------------------------------Create df obl info---------------------------------------------------------------------

df = pd.DataFrame()
for i in obl:
    r = req.get(url.format(i), json={"key": "value"}).json()
    r = pd.json_normalize(r['features'])
    df = pd.concat([df, r])
    
 #-------------------------Insert data from "df_obl_raion" into "[L0_X].[dbo].[AISGZK_OBLAST]"----------------------------------#

df = df.drop(['attributes.NAME_CNTR', 'attributes.NAME_ADM2', 'attributes.USL_DEFINITION', 'attributes.SHORT_NAME', 'attributes.LONG_NAME', 'attributes.CATEGORY', 'attributes.INPUTDATE'], axis=1)    
df = df.applymap(str)

insertQuery = '''
INSERT INTO [L0_X].[dbo].[AISGZK_OBLAST_GRANITSY]
(
   [ID],
   [NAME],
   [KATO],
   [POPULATION_AREA],
   [POPULATION_CITY],
   [POPULATION_VILLAGE],
   [GEOMETRY_RINGS]
)
VALUES (?, ?, ?, ?, ?, ?, ?)
'''
try:
    conn.autocommit = False
    for t in df.values.tolist():
        cursor.execute(insertQuery, t)
except Exception as e:
    print(t)
    cursor.rollback()
    print(e)
    print('transaction rolled back')
else:
    print('inserted successfully')
    conn.commit()
finally:
    conn.autocommit = True

,attributes.OBJECTID,attributes.NAME_ADM1,attributes.KATO,attributes.POPULATION_AREA,attributes.POPULATION_CITY,attributes.POPULATION_VILLAGE,geometry.rings
0,14,Акмолинская область,110000000,731328,341722,389606,"[[[7695992.550299998, 7096131.612800002], [771..."
0,11,Актюбинская область,150000000,786349,484960,301389,"[[[6909146.383900002, 6634551.627400003], [690..."
0,1,Алматинская область,190000000,1909362,443500,1465862,"[[[8549926.647100002, 5889590.236100003], [855..."
0,7,Атырауская область,230000000,542987,260396,282591,"[[[5994966.319899999, 6292921.5176], [5997235...."
0,2,Восточно-Казахстанская область,630000000,1395059,811588,583471,"[[[8608003.3992, 6233003.190499999], [8603759...."
0,3,Жамбылская область,310000000,1055976,410499,645477,"[[[8189497.182500001, 5780495.9353], [8244402...."
0,8,Западно-Казахстанская область,270000000,612551,297091,315460,"[[[5831017.3873, 6750352.0266999975], [5832104..."
0,18,Карагандинская область,350000000,1358064,1063394,294670,"[[[8583291.6048, 6379916.2663], [8583267.48070..."
0,9,Костанайская область,390000000,879579,446423,433156,"[[[7345380.997400001, 7290480.117299996], [734..."
0,5,Кызылординская область,430000000,712992,301615,411377,"[[[6925507.305199999, 6031799.336900003], [693..."


# geometry Районы

In [4]:
# ### Raion GEO

raion = []
df_features = pd.DataFrame()
r = req.post('https://aisgzk.kz/aisgzk/Index/BuildRegionTree', json={"key": "value"})
r=r.json()
data=r['data']
df_obl_raion=pd.DataFrame()
for i in data:
    obl=pd.json_normalize(i['children'])
    df_obl_raion = pd.concat([df_obl_raion, obl])
df_obl_raion['klkod'] = df_obl_raion['klkod'].astype(str)
df_obl_raion['klkod'] = df_obl_raion['klkod'].str.replace('|','', regex=True)
for k in df_obl_raion['klkod']:
    raion.append(k)
raion
url = "https://aisgzk.kz/aisgzk/Proxy/aisgzkTopo2/MapServer/34/query?f=json&where=RAION%20%3D%20%27{}%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&outSR=3857"
df_wkid = pd.DataFrame()
for i in raion:
    r = req.get(url.format(i), json={"key": "value"}).json()
    r = pd.json_normalize(r)
    df_wkid = pd.concat([df_wkid, r])
    df_features = pd.concat([df_features, r])
df_features['spatialReference.wkid'] = '102100'
df_features['spatialReference.latestWkid'] = '3857' ### add  wkid and latestWkid
df_features = df_features.rename(columns={'attributes.RAION' : 'RAION', 'geometry.rings':'geometry_rings'})
df_features = df_features.applymap(str)

# insert DB
# servername = '192.168.1.209'
dbname = 'L0_X'
engine = create_engine('mssql+pyodbc://@' + servername + '/' + dbname + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
df_features.to_sql('AISGZK_RAION_GEO', con = engine, if_exists='append')

From cffi callback <function _verify_callback at 0x0000028CD82773A0>:
Traceback (most recent call last):
  File "C:\Users\y.bukhar\miniconda3\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='aisgzk.kz', port=443): Max retries exceeded with url: /aisgzk/Proxy/aisgzkTopo2/MapServer/34/query?f=json&where=RAION%20%3D%20%2708124%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&outSR=3857 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

# get_bin_information

In [ ]:
servername = '192.168.1.209'
dbname = 'L0_X'

engine = create_engine('mssql+pyodbc://@' + servername + '/' + dbname + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [ ]:
#--------------------------SQL query to get kadastrs from DWH----------
get_kadastr = ''' 
SELECT DISTINCT REPLACE([KadastrovyyNomerZemelnogoUchastka],'-','') 
FROM(
  SELECT [KadastrovyyNomerZemelnogoUchastka]
  FROM [AA_DWH_X].[dbo].[L0_sev2021_flArea]
UNION ALL
  SELECT [KadastrovyyNomerZemelnogoUchastka]
  FROM [AA_DWH_X].[dbo].[L0_sev2020_flArea])a
'''
#--------------------------------------Create kadastr`s list----------------
df = pd.read_sql(get_kadastr, conn)
kadastrs = df.values.tolist()
kadastrs

#-----------------------------------------Create df_client_info and kadastrs with no information in AISGZK
url = 'https://aisgzk.kz/aisgzk/BINData/GetBINDataRoute'
empty_kadastrs_2 = []
df_bin_info_2 = pd.DataFrame()
for i in kadastrs:
    try:
        r = req.post(url, json={'kadNum':i,'language':'rus'})
        r = r.json()
        data = r['data']
        bin_info_json = pd.json_normalize(data).assign(kadastr=i)
        df_bin_info_2 = pd.concat([df_bin_info, bin_info_json])
        print(i)
    except:
        empty_kadastrs_2.append(i)
        pass
#-------------------------------------------------------Change columns order-----------------------------------------------------------#

df_bin_info_2 = df_bin_info_2[['bin', 'nameLast', 'nameFirst', 'nameMiddle', 'doczd_naim', 'nom', 'datev', 'docovd_naim', 'srok', 'osnova', 'kadastr']]

#-------------------------------------------INSERT INTRO L0_X.AISGZK_KADASTR_BIN_INFORMATSYA------------------------------------------#

df_bin_info_2.to_sql('AISGZK_KADASTR_BIN_INFORMATSYA', con = engine, if_exists='append', index=False)

# get_kadastr_info

In [ ]:
#--------------------------SQL query to get kadastrs from DWH
get_kadastr = ''' 
SELECT DISTINCT REPLACE([KadastrovyyNomerZemelnogoUchastka],'-','') 
FROM(
  SELECT [KadastrovyyNomerZemelnogoUchastka]
  FROM [AA_DWH_X].[dbo].[L0_sev2021_flArea]
  --WHERE KadastrovyyNomerZemelnogoUchastka <> '010017778888'
UNION ALL
  SELECT [KadastrovyyNomerZemelnogoUchastka]
  FROM [AA_DWH_X].[dbo].[L0_sev2020_flArea])a
--WHERE KadastrovyyNomerZemelnogoUchastka <> '01-001-777-8888'  
'''
df = pd.read_sql(get_kadastr, conn)
kadastrs = df.values.tolist()
kadastrs

#----------------------------Create df_client_info and kadastrs with no information in AISGZK

empty_kadastrs = []
df_client_info = pd.DataFrame()
for i in kadastrs:
    try:
        r = req.post('https://aisgzk.kz/aisgzk/Index/FindObjInfoForMap', json={'kadNum':i[0],'languages':'Ru'})
        r = r.json()
        client_info_json = pd.json_normalize(r)\
                             .drop(['Name.Kz'], axis=1)\
                             .transpose() 
        df_client_info = pd.concat([df_client_info, client_info_json])
    except:
        empty_kadastrs.append(i)
        pass
df_client_info


#----------------------------------------------------RENAME COLUMN_NAMES 

df_client_info = df.rename(columns={'Кадастровый номер':'KADASTR','Предоставленное право':'PREDOSTAVLENNOE_PRAVO','Срок землепользования':'SROK_ZEMLEPOLZOVANYA','Категория земель':'KATEGORYA_ZEMEL','Целевое назначение':'TSELEVOE_NAZNACHENIE','Местоположение':'MESTOPOLOZHENIE','Площадь (кв.м.)':'PLOSHAD','Кадастровая оценка':'KADASTROVAYA_OTSENKA','Землепользователи':'ZEMLEPOLZOVATELI','Делимый участок?':'DELIMIY_UCHASTOK','Ограничения':'OGRANICHENIYA'})

#-----------------------------------------------------INSERT INTO L0_X.AISGZK_KADASTR_ZEMLYA_INFORMATSYA

df.to_sql('AISGZK_KADASTR_ZEMLYA_INFORMATSYA', con = engine, if_exists='append', index=False)




# Get_cadastr_Polygon

In [ ]:
# ----------------------------pypyodbc используется для загрузки болших данных стобца
import pypyodbc
conn = pypyodbc.connect('Driver=SQL Server;Server=192.168.1.209;Database=L0_X;Trusted_Connection=yes;')
cursor = conn.cursor()

In [23]:
#--------------------------SQL query to get kadastrs and ID OBL_Raion from DWH
kadstr=pd.read_sql('''SELECT 
      distinct[kadastr]
	  ,AR.LAYERS_ID
  FROM [L0_X].[dbo].[AISGZK_KADASTR_BIN_INFORMATSYA_2]

LEFT JOIN L0_X.dbo.AISGZK_RAION AR 
  ON REPLACE(AR.KLKOD, '|', '') =  LEFT(REPLACE([kadastr],'-',''), 5)

  where [bin]<>''
  and [kadastr] not in (SELECT 
      [value]
  FROM [L0_X].[dbo].[AISGZK_KADASTR_POLYGON])

''' ,conn)

C:\Users\y.bukhar\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# func insert DB
def insert_into (df):
   #-----------------------------------------------------INSERT INTO L0_X.AISGZK_KADASTR_ZEMLYA_INFORMATSYA
    insertQuery = '''
    INSERT INTO [dbo].[AISGZK_KADASTR_POLYGON]
               ([layerId]
               ,[layerName]
               ,[displayFieldName]
               ,[value]
               ,[attributes_OBJECTID]
               ,[attributes_Shape]
               ,[attributes_KAD_NOMER]
               ,[attributes_KOORDINATED_UP]
               ,[attributes_Shape_Length]
               ,[attributes_Shape_Area]
               ,[geometryType]
               ,[geometry_rings]
               ,[geometry_spatialReference_wkid]
               ,[geometry_spatialReference_latestWkid]
               )
         VALUES
               ( ?,?,?,?,?,?,?,?,?,?,
                ?,?,?,?
                );
    '''
    #i = 0
    try:
        conn.autocommit = False
        for t in df.values.tolist():
            cursor.execute(insertQuery, t)
    except Exception as e:
        print(t)
        cursor.rollback()
        print(e)
        print('transaction rolled back')
    else:
        print('inserted successfully')
        conn.commit()
    finally:
        conn.autocommit = True
#         print(i[0]) 
# print("Mades")

In [ ]:
# -------------Get Polygon KX TOO FX
url = 'https://aisgzk.kz/aisgzk/Proxy/aisgzkZem2/MapServer/find?f=json&searchText={}&contains=false&returnGeometry=true&layers={}&searchFields=KAD_NOMER&sr=3857'


for i in kadstr.values.tolist():
    df=[]
    date=[]
    null_kadastr=[]
    r = req.get(url.format(i[0], i[1]), json={"key": "value"}).json()
#     r = pd.json_normalize(r['results'])
    print(i[0])
    try:
        if 'SHAPE' in r['results'][0]['attributes']:
            date.append(r['results'][0]['layerId'])
            date.append(r['results'][0]['layerName'])
            date.append(r['results'][0]['displayFieldName'])
            date.append(r['results'][0]['value'])
            date.append(r['results'][0]['attributes']['OBJECTID'])
            date.append(r['results'][0]['attributes']['SHAPE'])
            date.append(r['results'][0]['attributes']['KAD_NOMER'])
            date.append(r['results'][0]['attributes']['koordinated'])
            date.append(r['results'][0]['attributes']['SHAPE_Length'])
            date.append(r['results'][0]['attributes']['SHAPE_Area'])
            date.append(r['results'][0]['geometryType'])
            date.append(r['results'][0]['geometry']['rings'])
            date.append(r['results'][0]['geometry']['spatialReference']['wkid'])
            date.append(r['results'][0]['geometry']['spatialReference']['latestWkid'])
        elif 'Shape' in r['results'][0]['attributes']:
            date.append(r['results'][0]['layerId'])
            date.append(r['results'][0]['layerName'])
            date.append(r['results'][0]['displayFieldName'])
            date.append(r['results'][0]['value'])
            date.append(r['results'][0]['attributes']['OBJECTID'])
            date.append(r['results'][0]['attributes']['Shape'])
            date.append(r['results'][0]['attributes']['KAD_NOMER'])
            date.append(r['results'][0]['attributes']['KOORDINATED'])
            date.append(r['results'][0]['attributes']['Shape_Length'])
            date.append(r['results'][0]['attributes']['Shape_Area'])
            date.append(r['results'][0]['geometryType'])
            date.append(r['results'][0]['geometry']['rings'])
            date.append(r['results'][0]['geometry']['spatialReference']['wkid'])
            date.append(r['results'][0]['geometry']['spatialReference']['latestWkid'])
            df.append(date)
        df = pd.DataFrame(df)
        df[[0,12,13,11]]=df[[0,12,13,11]].applymap(str)
        insert_into (df)
        print(i[0])
    except:
        null_kadastr.append(i[0])
#     df